In [12]:
%load_ext autoreload
%autoreload 2
from data_loader import load_datasets, create_data_loaders
import torch
import os
from cnn_model import Net as init_cnn
import numpy as np
from init_nets import init_resnet, init_efficientnet
import torch.nn.functional as F
from serialization import save, load

SEED = 42

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# model paths
cnn_model_path = "output/models/cnn/cnn"
resnet_model_path = "output/models/resnet/resnet"
efficientnet_model_path = "output/models/efficientnet/efficientnet"
dropout = 0.4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dir_ensemble = "output/history/ensemble/"
path_ensemble_hard = "output/history/ensemble/history_hard.pkl"
path_ensemble_soft = "output/history/ensemble/history_soft.pkl"

if not os.path.exists(dir_ensemble):
    os.makedirs(dir_ensemble)

In [14]:
batch_size = 128
train, val, test = load_datasets((32, 32))
_, _, test_dataloader = create_data_loaders(train, val, test, batch_size)

labels = torch.tensor(test.targets).to(device)

In [32]:
def get_proba(model, dataloader, device):
    all_outputs = []
    model.eval()
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            output = model(x)
            output = F.softmax(output, dim=1)
            all_outputs.append(output)
    return torch.cat(all_outputs, dim=0)

def get_prediction(proba):
    _, predicted = torch.max(proba, 1)
    return predicted

def compute_accuracy(preds, labels):
    accuracy = preds.eq(labels).sum().item() / labels.size(0)
    return accuracy


In [7]:
# ensemble models
hard_accuracy_list = []
soft_accuracy_list = []

for i in range(1, 6):
    # set paths
    suffix = f"_{i}.pth"
    cnn_model_path_i = cnn_model_path + suffix
    resnet_model_path_i = resnet_model_path + suffix
    efficientnet_model_path_i = efficientnet_model_path + suffix
    
    # initialize nets
    cnn = init_cnn(dropout=dropout).to(device)
    resnet = init_resnet().to(device)
    efficientnet = init_efficientnet().to(device)
    
    # load weights
    cnn.load_state_dict(torch.load(cnn_model_path_i, weights_only=True))
    resnet.load_state_dict(torch.load(resnet_model_path_i, weights_only=True))
    efficientnet.load_state_dict(torch.load(efficientnet_model_path_i, weights_only=True))
    
    # get probabilities
    cnn_prob = get_proba(cnn, test_dataloader, device)
    resnet_prob = get_proba(resnet, test_dataloader, device)
    efficientnet_prob = get_proba(efficientnet, test_dataloader, device)
    
    # hard voting
    cnn_pred = get_prediction(cnn_prob)
    resnet_pred = get_prediction(resnet_prob)
    efficientnet_pred = get_prediction(efficientnet_prob)
    
    models_pred = torch.stack([cnn_pred, resnet_pred, efficientnet_pred], dim=0)
    hard_pred, _ = torch.mode(models_pred, dim=0)
    hard_accuracy = compute_accuracy(hard_pred, labels)
    hard_accuracy_list.append(hard_accuracy)
    
    # soft voting
    ensemble_prob = (cnn_prob + resnet_prob + efficientnet_prob) / 3
    soft_pred = get_prediction(ensemble_prob)
    soft_accuracy = compute_accuracy(soft_pred, labels)
    soft_accuracy_list.append(soft_accuracy)

# saving results
save(np.array(hard_accuracy_list), path_ensemble_hard)
save(np.array(soft_accuracy_list), path_ensemble_soft)

In [8]:
print(hard_accuracy_list)
print(soft_accuracy_list)

[34430, 16900, 30118, 23718, 19577]
[55894, 51004, 52562, 55671, 53152]


### Tests

In [48]:
from training_functions import evaluate
import torch.nn as nn

resnet_model_path_i = "output/models/resnet/resnet_1.pth"
resnet = init_resnet().to(device)
resnet.load_state_dict(torch.load(resnet_model_path_i, weights_only=True))
accuracy, avg_loss = evaluate(resnet, test_dataloader, nn.CrossEntropyLoss(), device)
print(accuracy)
print(avg_loss)

16.68111111111111
2.3059331046210393e-05


In [49]:
resnet_model_path_i = "output/models/resnet/resnet_2.pth"
resnet = init_resnet().to(device)
resnet.load_state_dict(torch.load(resnet_model_path_i, weights_only=True))
accuracy, avg_loss = evaluate(resnet, test_dataloader, nn.CrossEntropyLoss(), device)
print(accuracy)
print(avg_loss)

16.263333333333332
1.809429195192125e-05


In [47]:
cnn_model_path_i = "output/models/cnn/cnn_1.pth"
cnn = init_cnn(dropout).to(device)
cnn.load_state_dict(torch.load(cnn_model_path_i, weights_only=True))
accuracy, avg_loss = evaluate(cnn, test_dataloader, nn.CrossEntropyLoss(), device)
print(accuracy)
print(avg_loss)

63.66777777777778
4.586939348114861e-06
